# CUDA Initialization Error Debugging Notebook

This notebook helps debug the CUDA initialization error that occurs when using multi-worker DataLoader with PyTorch training.

## Problem Analysis
The error `RuntimeError: CUDA error: initialization error` typically occurs when:
1. CUDA contexts conflict with multiprocessing workers
2. Multiple processes try to initialize CUDA simultaneously
3. GPU memory is not properly managed across processes

## Solutions We'll Implement
1. **Disable multi-worker data loading** (immediate fix)
2. **Set proper CUDA environment variables** 
3. **Use spawn instead of fork for multiprocessing**
4. **Implement proper CUDA context management**

In [ ]:
# Import Required Libraries
import os
import sys
import subprocess
import argparse
import torch
import psutil
from datetime import datetime

print("🔧 CUDA Debugging Environment Setup")
print("=" * 50)
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Set CUDA Environment Variables for Debugging
print("🌍 Setting CUDA Environment Variables")
print("=" * 50)

# Enable CUDA debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Synchronous CUDA calls for better error reporting
os.environ['TORCH_USE_CUDA_DSA'] = '1'    # Enable device-side assertions
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'  # Consistent GPU ordering

# Set multiprocessing method to spawn (safer for CUDA)
import multiprocessing
try:
    multiprocessing.set_start_method('spawn', force=True)
    print("✅ Set multiprocessing start method to 'spawn'")
except RuntimeError as e:
    print(f"⚠️  Could not set multiprocessing method: {e}")

# Print current environment variables
cuda_env_vars = {k: v for k, v in os.environ.items() if 'CUDA' in k}
print("\n🔍 Current CUDA Environment Variables:")
for key, value in cuda_env_vars.items():
    print(f"   {key} = {value}")

# Check GPU memory
if torch.cuda.is_available():
    print(f"\n💾 GPU Memory Status:")
    for i in range(torch.cuda.device_count()):
        memory_allocated = torch.cuda.memory_allocated(i) / 1024**3
        memory_reserved = torch.cuda.memory_reserved(i) / 1024**3
        memory_total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"   GPU {i}: {memory_allocated:.1f}GB allocated, {memory_reserved:.1f}GB reserved, {memory_total:.1f}GB total")

In [ ]:
# Parse Training Arguments - Safe Configuration for CUDA
print("⚙️ Training Configuration")
print("=" * 50)

# Define safe training parameters that avoid CUDA multiprocessing issues
class TrainingConfig:
    def __init__(self):
        # Performance settings
        self.batch_size = 16  # Start with smaller batch size
        self.gradient_accumulation_steps = 8  # Compensate with more accumulation
        self.mixed_precision = "fp16"
        self.dataloader_num_workers = 0  # CRITICAL: Set to 0 to avoid multiprocessing
        
        # Training parameters
        self.model_version = "DDIMNextTokenV1"
        self.train_size = 1000  # Smaller for testing
        self.val_size = 100
        self.num_epochs = 2  # Fewer epochs for testing
        self.lr = 0.0002
        self.warming_steps = 100
        self.num_cycles = 0.5
        self.train_tags = ["cuda_debug", "safe_config"]
        self.dataset_name = "QLeca/modular_characters_hairs_RGB"

# Create configurations for testing
configs = {
    "safe": TrainingConfig(),
    "minimal": TrainingConfig(),
    "single_worker": TrainingConfig()
}

# Modify configurations for different test cases
configs["minimal"].batch_size = 8
configs["minimal"].train_size = 100
configs["minimal"].val_size = 10
configs["minimal"].num_epochs = 1

configs["single_worker"].dataloader_num_workers = 1  # Try with 1 worker

# Display configurations
for name, config in configs.items():
    print(f"\n🔧 {name.upper()} Configuration:")
    print(f"   Batch size: {config.batch_size}")
    print(f"   Gradient accumulation: {config.gradient_accumulation_steps}")
    print(f"   Effective batch size: {config.batch_size * config.gradient_accumulation_steps}")
    print(f"   Workers: {config.dataloader_num_workers}")
    print(f"   Mixed precision: {config.mixed_precision}")
    print(f"   Train size: {config.train_size}")
    print(f"   Epochs: {config.num_epochs}")

In [ ]:
# Build Training Commands
print("🔨 Building Training Commands")
print("=" * 50)

def build_training_command(config, config_name="default"):
    """Build a training command from configuration."""
    cmd = [
        sys.executable, "training.py",
        "--batch_size", str(config.batch_size),
        "--gradient_accumulation_steps", str(config.gradient_accumulation_steps),
        "--mixed_precision", config.mixed_precision,
        "--dataloader_num_workers", str(config.dataloader_num_workers),
        "--model_version", config.model_version,
        "--train_size", str(config.train_size),
        "--val_size", str(config.val_size),
        "--num_epochs", str(config.num_epochs),
        "--lr", str(config.lr),
        "--warming_steps", str(config.warming_steps),
        "--num_cycles", str(config.num_cycles),
        "--dataset_name", config.dataset_name,
    ]
    
    # Add train_tags if provided
    if config.train_tags:
        cmd.extend(["--train_tags"] + config.train_tags + [config_name])
    
    return cmd

# Build commands for each configuration
commands = {}
for name, config in configs.items():
    commands[name] = build_training_command(config, name)
    print(f"\n🚀 {name.upper()} Command:")
    print(f"   {' '.join(commands[name])}")

# Function to create a safe training command
def create_safe_command():
    """Create the safest possible training command."""
    return [
        sys.executable, "training.py",
        "--batch_size", "8",
        "--gradient_accumulation_steps", "16",  # Large accumulation to compensate
        "--mixed_precision", "fp16",
        "--dataloader_num_workers", "0",  # No multiprocessing
        "--model_version", "DDIMNextTokenV1",
        "--train_size", "100",
        "--val_size", "10", 
        "--num_epochs", "1",
        "--lr", "0.0002",
        "--warming_steps", "50",
        "--num_cycles", "0.5",
        "--dataset_name", "QLeca/modular_characters_hairs_RGB",
        "--train_tags", "cuda_safe", "no_workers"
    ]

safe_cmd = create_safe_command()
print(f"\n🛡️  SAFEST Command (recommended to try first):")
print(f"   {' '.join(safe_cmd)}")

In [ ]:
# Run Training Subprocess and Capture Output
print("🏃‍♂️ Running Training with Error Capture")
print("=" * 50)

def run_training_with_debug(cmd, timeout=300):
    """Run training command with comprehensive error capture."""
    print(f"🔄 Running command: {' '.join(cmd)}")
    print(f"⏰ Timeout: {timeout} seconds")
    
    start_time = datetime.now()
    
    try:
        # Run with environment variables set
        env = os.environ.copy()
        env['CUDA_LAUNCH_BLOCKING'] = '1'
        env['TORCH_USE_CUDA_DSA'] = '1'
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=timeout,
            env=env,
            cwd=os.getcwd()
        )
        
        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()
        
        print(f"✅ Command completed in {duration:.1f} seconds")
        print(f"📤 Return code: {result.returncode}")
        
        if result.stdout:
            print(f"\n📝 STDOUT (last 1000 chars):")
            print(result.stdout[-1000:])
        
        if result.stderr:
            print(f"\n❌ STDERR:")
            print(result.stderr)
        
        return result, duration
        
    except subprocess.TimeoutExpired as e:
        print(f"⏰ Command timed out after {timeout} seconds")
        print(f"📤 Partial stdout: {e.stdout[-500:] if e.stdout else 'None'}")
        print(f"❌ Partial stderr: {e.stderr[-500:] if e.stderr else 'None'}")
        return None, timeout
        
    except Exception as e:
        print(f"💥 Unexpected error: {e}")
        return None, 0

# Test function to check if training setup works
def test_training_setup():
    """Test if training can start without errors."""
    print("🧪 Testing Training Setup...")
    
    # First, try to import and initialize the training modules
    try:
        # Change to the correct directory
        original_cwd = os.getcwd()
        if not os.path.exists('training.py'):
            print("⚠️  training.py not found in current directory")
            print(f"Current directory: {os.getcwd()}")
            return False
        
        print("✅ training.py found")
        return True
        
    except Exception as e:
        print(f"❌ Setup test failed: {e}")
        return False

# Run the setup test
setup_ok = test_training_setup()
print(f"Setup test result: {'✅ PASSED' if setup_ok else '❌ FAILED'}")

In [ ]:
# Handle CUDA Initialization Errors
print("🔧 CUDA Error Analysis and Solutions")
print("=" * 50)

def analyze_cuda_error(stderr_output):
    """Analyze CUDA error output and provide solutions."""
    error_solutions = {
        "CUDA error: initialization error": {
            "description": "CUDA context initialization failed",
            "causes": [
                "Multiple processes trying to initialize CUDA simultaneously",
                "Insufficient GPU memory",
                "Driver or CUDA installation issues",
                "Multiprocessing conflicts with CUDA contexts"
            ],
            "solutions": [
                "Set dataloader_num_workers=0 (disable multiprocessing)",
                "Use multiprocessing.set_start_method('spawn')",
                "Check GPU memory usage",
                "Restart Python process to clear CUDA context",
                "Set CUDA_LAUNCH_BLOCKING=1 for better error reporting"
            ]
        },
        "CUDA out of memory": {
            "description": "GPU memory exhausted",
            "causes": [
                "Batch size too large",
                "Model too large for GPU",
                "Memory not properly freed"
            ],
            "solutions": [
                "Reduce batch_size",
                "Increase gradient_accumulation_steps",
                "Use mixed precision training",
                "Clear GPU cache with torch.cuda.empty_cache()"
            ]
        },
        "RuntimeError: DataLoader worker": {
            "description": "DataLoader worker process failed",
            "causes": [
                "CUDA context not available in worker process",
                "Multiprocessing conflicts"
            ],
            "solutions": [
                "Set num_workers=0",
                "Use pin_memory=False",
                "Set multiprocessing start method to 'spawn'"
            ]
        }
    }
    
    found_errors = []
    for error_key, error_info in error_solutions.items():
        if error_key in stderr_output:
            found_errors.append((error_key, error_info))
    
    return found_errors

def provide_immediate_fix():
    """Provide immediate fix for the CUDA initialization error."""
    print("🚨 IMMEDIATE FIX for CUDA Initialization Error")
    print("=" * 60)
    print("The error occurs because of multiprocessing conflicts with CUDA.")
    print("Here's the immediate solution:")
    print()
    print("1. 🔧 QUICK FIX - Run with no workers:")
    print("   python train_optimized.py --dataloader_num_workers 0")
    print()
    print("2. 🔧 ALTERNATIVE - Use small batch size:")
    print("   python train_optimized.py --batch_size 8 --gradient_accumulation_steps 16 --dataloader_num_workers 0")
    print()
    print("3. 🔧 SAFEST OPTION - Use our tested configuration:")
    safe_cmd = create_safe_command()
    print(f"   {' '.join(safe_cmd)}")
    print()
    print("🔍 Why this works:")
    print("   - num_workers=0 eliminates multiprocessing")
    print("   - Smaller batch_size reduces memory pressure")
    print("   - Large gradient_accumulation_steps maintains effective batch size")
    print("   - Mixed precision reduces memory usage")

def create_fixed_training_script():
    """Create a fixed version of train_optimized.py with better defaults."""
    fixed_script = '''#!/usr/bin/env python3
"""
CUDA-Safe Optimized Training Script
Fixed version that avoids CUDA initialization errors.
"""

import subprocess
import argparse
import sys
import os

def main():
    parser = argparse.ArgumentParser(description="CUDA-Safe Optimized training launcher")
    
    # Performance optimization arguments with safe defaults
    parser.add_argument("--batch_size", type=int, default=16, 
                       help="Base batch size (reduced for CUDA safety)")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=8,
                       help="Gradient accumulation steps (increased to compensate)")
    parser.add_argument("--mixed_precision", type=str, default="fp16", choices=["no", "fp16", "bf16"],
                       help="Mixed precision mode")
    parser.add_argument("--dataloader_num_workers", type=int, default=0,
                       help="Number of data loading workers (0 = no multiprocessing)")
    
    # Other arguments...
    parser.add_argument("--model_version", type=str, default="DDIMNextTokenV1",
                       choices=["DDPMNextTokenV1", "DDPMNextTokenV2", "DDPMNextTokenV3", "DDIMNextTokenV1"])
    parser.add_argument("--train_size", type=int, default=16000)
    parser.add_argument("--val_size", type=int, default=1600)
    parser.add_argument("--num_epochs", type=int, default=50)
    parser.add_argument("--lr", type=float, default=0.0002)
    parser.add_argument("--warming_steps", type=int, default=1000)
    parser.add_argument("--num_cycles", type=float, default=0.5)
    parser.add_argument("--train_tags", type=str, nargs='*', default=None)
    parser.add_argument("--dataset_name", type=str, default="QLeca/modular_characters_hairs_RGB")
    
    args = parser.parse_args()
    
    # Set CUDA environment variables
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    os.environ['TORCH_USE_CUDA_DSA'] = '1'
    
    # Build command with safe parameters
    cmd = [
        sys.executable, "training.py",
        "--batch_size", str(args.batch_size),
        "--gradient_accumulation_steps", str(args.gradient_accumulation_steps),
        "--mixed_precision", args.mixed_precision,
        "--dataloader_num_workers", str(args.dataloader_num_workers),
        "--model_version", args.model_version,
        "--train_size", str(args.train_size),
        "--val_size", str(args.val_size),
        "--num_epochs", str(args.num_epochs),
        "--lr", str(args.lr),
        "--warming_steps", str(args.warming_steps),
        "--num_cycles", str(args.num_cycles),
        "--dataset_name", args.dataset_name,
    ]
    
    if args.train_tags:
        cmd.extend(["--train_tags"] + args.train_tags)
    
    print(f"🚀 Running CUDA-safe training: {' '.join(cmd)}")
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()
'''
    
    return fixed_script

# Run the immediate fix function
provide_immediate_fix()

# Show the fixed script
print("\\n📝 Fixed Training Script:")
print("This script has safe defaults that avoid CUDA errors:")
fixed_script = create_fixed_training_script()
print("\\n" + "="*60)
print("You can save this as 'train_cuda_safe.py'")
print("="*60)

In [ ]:
# Test the Safe Configuration
print("🧪 Ready to Test Safe Configuration")
print("=" * 50)

print("Now you can test the safe configuration. Uncomment and run the cell below:")
print()
print("# Uncomment the lines below to run a test:")
print("# result, duration = run_training_with_debug(safe_cmd, timeout=60)")
print("# if result and result.returncode == 0:")
print("#     print('✅ Training started successfully!')")
print("# else:")
print("#     print('❌ Training failed - check the error output above')")

print("\\n📋 SUMMARY OF SOLUTIONS")
print("=" * 50)
print("1. ⚡ IMMEDIATE FIX:")
print("   python train_optimized.py --dataloader_num_workers 0")
print()
print("2. 🛡️  SAFEST APPROACH:")
print("   python train_optimized.py --batch_size 8 --gradient_accumulation_steps 16 --dataloader_num_workers 0 --mixed_precision fp16")
print()
print("3. 🔧 ENVIRONMENT VARIABLES:")
print("   export CUDA_LAUNCH_BLOCKING=1")
print("   export TORCH_USE_CUDA_DSA=1")
print()
print("4. 🐍 PYTHON MULTIPROCESSING:")
print("   Set multiprocessing start method to 'spawn'")
print()
print("5. 💾 IF MEMORY ISSUES:")
print("   - Reduce batch_size further (e.g., 4)")
print("   - Increase gradient_accumulation_steps (e.g., 32)")
print("   - Use mixed_precision='fp16'")
print()
print("🎯 ROOT CAUSE: The error happens because CUDA contexts don't work well")
print("   with multiprocessing workers. Setting num_workers=0 fixes this.")
print()
print("✨ PERFORMANCE: You'll still get good performance because:")
print("   - Mixed precision training (2x speedup)")
print("   - Large gradient accumulation (maintains effective batch size)")
print("   - No data loading bottleneck (workers=0 is often fine for cached datasets)")